## Loading the Dataset with Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, from_json
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType, ArrayType

# Initialize Spark session
spark = SparkSession.builder \
            .appName("BiggaDataSpark2") \
            .config("spark.driver.memory", "4g") \
            .getOrCreate() 

# Define the schema for the metadata and review JSON structure
schema = StructType([
    StructField("metadata", StructType([
        StructField("main_category", StringType(), True),
        StructField("title", StringType(), True),
        StructField("average_rating", FloatType(), True),
        StructField("rating_number", IntegerType(), True),
        StructField("features", StringType(), True),
        StructField("description", StringType(), True),
        StructField("price", FloatType(), True),
        StructField("image_count", IntegerType(), True),
        StructField("video_count", IntegerType(), True),
        StructField("store", StringType(), True),
        StructField("details", StringType(), True),
        # StructField("parent_asin", StringType(), True)
    ]), True),
    StructField("review", StructType([
        StructField("rating", FloatType(), True),
        StructField("text", StringType(), True),
        StructField("images", IntegerType(), True),
        # StructField("parent_asin", StringType(), True),
        StructField("user_id", StringType(), True),
        StructField("timestamp", StringType(), True),
        StructField("helpful_vote", IntegerType(), True),
        StructField("verified_purchase", BooleanType(), True)
    ]), True)
])

# Read the file as text (each line is a key-value pair)
file_path = "output_Final_Merged_Dataset/part-r-00000"
df = spark.read.text(file_path)

# Split the line into the key and the JSON value (assuming a tab separates key and value)
df_split = df.select(split(col("value"), "\t").alias("key_value"))

# Extract the key and the JSON part (JSON starts from the second part of the split)
df_json = df_split.select(col("key_value")[0].alias("parent_asin"), col("key_value")[1].alias("json_value"))

# Parse the JSON value using the schema defined above
df_parsed = df_json.withColumn("parsed_json", from_json(col("json_value"), schema))

# Select individual columns from the parsed JSON
df_final = df_parsed.select("parent_asin", "parsed_json.metadata", "parsed_json.review")


25/03/21 18:12:56 WARN Utils: Your hostname, AA.local resolves to a loopback address: 127.0.0.1; using 172.20.10.6 instead (on interface en0)
25/03/21 18:12:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/21 18:12:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Flatten the metadata and review fields
df_merged = df_final.select(
    "parent_asin",  # Keep the parent_asin column
    "metadata.*",    # Flatten all fields from the metadata JSON
    "review.*"       # Flatten all fields from the review JSON
)

# Show the result
df_merged.show(truncate=False)

+-----------+-------------+----------------------------------------------------------------------------------------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Quick EDA on the Merged Dataset

In [3]:
# Describe the DataFrame (numeric columns only)
df_merged.describe().show(truncate=False)


25/03/21 18:13:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+---------------+---------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# Print the schema (column names and types)
df_merged.printSchema()


root
 |-- parent_asin: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- average_rating: float (nullable = true)
 |-- rating_number: integer (nullable = true)
 |-- features: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: float (nullable = true)
 |-- image_count: integer (nullable = true)
 |-- video_count: integer (nullable = true)
 |-- store: string (nullable = true)
 |-- details: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- text: string (nullable = true)
 |-- images: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- helpful_vote: integer (nullable = true)
 |-- verified_purchase: boolean (nullable = true)



In [ ]:
df_sentiment_analysis = df_merged.select("parent_asin", "main_category", "rating", "text")

df_sentiment_analysis.show(truncate=False)

In [5]:
from pyspark.sql import functions as F
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Aggregate the 'text' field (reviews) for each main_category by collecting the texts into a list
df_with_reviews = df_merged.groupBy("main_category").agg(F.concat_ws(" ", F.collect_list("text")).alias("all_reviews"))

# Collect the data for word cloud generation
grouped_data = df_with_reviews.collect()

# Create a word cloud for each main_category
for row in grouped_data:
    main_category = row['main_category']
    reviews_text = row['all_reviews']
    
    # Generate word cloud for this category
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(reviews_text)
    
    # Plot the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Word Cloud for {main_category}")
    plt.show()


25/03/21 18:18:52 ERROR Executor: Exception in task 4.0 in stage 6.0 (TID 218)6]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.codegen.BufferHolder.grow(BufferHolder.java:80)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.grow(UnsafeWriter.java:63)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.writeUnalignedBytes(UnsafeWriter.java:127)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.write(UnsafeWriter.java:110)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$generateResultProjection$5(AggregationIterator.scala:260)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator$$Lambda$3748/952923458.apply(Unknown Source)
	at org.apache.spark.sql.execution.aggregate.ObjectAggregationIterator.next(ObjectAggregationIterator.scala:97)
	at org.apache.spar

Py4JJavaError: An error occurred while calling o79.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 6.0 failed 1 times, most recent failure: Lost task 4.0 in stage 6.0 (TID 218) (172.20.10.6 executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.codegen.BufferHolder.grow(BufferHolder.java:80)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.grow(UnsafeWriter.java:63)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.writeUnalignedBytes(UnsafeWriter.java:127)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.write(UnsafeWriter.java:110)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$generateResultProjection$5(AggregationIterator.scala:260)
	at org.apache.spark.sql.execution.aggregate.AggregationIter

## Doing Sentiment Analysis

### Loading the RoBERTa Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm.notebook import tqdm

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/Users/alan/Desktop/Year 3 Tri 2/INF2006 Cloud Computing and Big Data/BigDataProject/ARA/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def roberta_polarity_scores(sentence):
    encoded_text = tokenizer(sentence, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['Text']
        myid = row['Id']
        vader_result = SIA.polarity_scores(text)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f"vader_{key}"] = value
        roberta_result = roberta_polarity_scores(text)
        both = {**vader_result_rename, **roberta_result}
        res[myid] = both
    except RuntimeError:
        print(f'Broke at id {myid}')


results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'Id'})
results_df = results_df.merge(df, how='left')

AttributeError: 'DataFrame' object has no attribute 'iterrows'